https://www.kaggle.com/code/cluysedylan/scrabbling-with-linear-regression/edit
https://spark.apache.org/docs/latest/ml-guide.html

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv('/kaggle/input/scrabble-player-rating/train.csv')
test  = pd.read_csv('/kaggle/input/scrabble-player-rating/test.csv')
turns = pd.read_csv('/kaggle/input/scrabble-player-rating/turns.csv')
games = pd.read_csv('/kaggle/input/scrabble-player-rating/games.csv')
sample_submission = pd.read_csv('/kaggle/input/scrabble-player-rating/sample_submission.csv')

In [ ]:
brief_df = pd.concat([train,test], axis=0)
brief_df = brief_df.sort_values(['game_id'])
bots     = ['BetterBot', 'STEEBot', 'HastyBot']
user_df  = brief_df[~brief_df['nickname'].isin(bots)]
user_df  = user_df.rename(columns={'nickname': 'user_name', 'score': 'user_score', 'rating': 'user_rating'})
bot_df   = brief_df[brief_df['nickname'].isin(bots)]
bot_df   = bot_df.rename(columns={'nickname': 'bot_name', 'score': 'bot_score', 'rating': 'bot_rating'})
main_df  = pd.merge(user_df, bot_df, on='game_id')
main_df.head()

In [ ]:
main_df['user_freq'] = main_df.groupby('user_name')['user_name'].transform('count')
encode_bots          = LabelEncoder()
main_df['bot_name']  = encode_bots.fit_transform(main_df['bot_name'])
main_df.head()

In [ ]:
train_df = main_df[~main_df['user_rating'].isna()].reset_index(drop=True)
train_df.head()

In [ ]:
test_df  = main_df[main_df['user_rating'].isna()].reset_index(drop=True)
test_df.head()

In [ ]:
X = train_df.drop(['user_name', 'user_rating'], axis=1)
y = train_df['user_rating'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)
model = LinearRegression()
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [ ]:
MSE  = mean_squared_error(y_hat, y_test)
RMSE = np.sqrt(MSE)
RMSE

In [ ]:
model = LinearRegression()
model.fit(X, y)
test_df['user_rating'] = model.predict(test_df.drop(['user_name', 'user_rating'], axis=1))
final_sub = test_df[['game_id', 'user_rating']]
final_sub = final_sub.rename(columns={'user_rating': 'rating'})
final_sub.to_csv('sub_0.csv', index=False)